# Calculating counter term $C_0$ for j=0

In [2]:
%run LS.ipynb

<IPython.core.display.Javascript object>

## inputs for method 1

In [3]:
def method_input(phase_list,q0):
    
    index_strong = 0
    Vtensor0_strong = Vtensor_calc(phase_list,index_strong)
    Vtensorq0_strong = Vtensor_q0_calc(phase_list,Vtensor0_strong,q0,index_strong)
    
    index_odd = 1
    Vtensor0_odd = Vtensor_calc(phase_list,index_odd)
    Vtensorq0_odd = Vtensor_q0_calc(phase_list,Vtensor0_odd,q0,index_odd)
    
    index_even_odd = 2
    Vtensor0_even_odd = Vtensor_calc(phase_list,index_even_odd)
    Vtensorq0_even_odd = Vtensor_q0_calc(phase_list,Vtensor0_even_odd,q0,index_even_odd)
    
    return [Vtensorq0_strong, Vtensorq0_odd,Vtensorq0_even_odd]
    
    

In [4]:
# # Example

# phase_list = phase_dict[0]
# p_wave = [phase_name(*phase) for phase in phase_list]
# print(phase_list,'==',p_wave)

# q0 = 100

# [Vtensorq0_strong, Vtensorq0_odd,Vtensorq0_even_odd] = method_input(phase_list,q0)

# print('Vtensorq0_strong:\n')
# print_(Vtensorq0_strong)

# print('\nVtensorq0_odd:\n')
# print_(Vtensorq0_odd)

# print('\nVtensorq0_even_odd:\n')
# print_(Vtensorq0_even_odd)

## $ \sum_{\alpha_1}F_n^{\alpha',\alpha_1, \alpha}(p',p)$


 $ \large sum\_1 =  2 \mu \int dp \frac{T_s p^2 V_{odd} - T_s q^2 V_{odd}}{q^2 -p^2}$
 
 $\large term\_1 = 2 \mu I_\Lambda T_s(p',q) V_{odd}(q,p)$
 
 $\large term\_2 = i \mu \pi q T_s(p',q) V_{odd}(q,p)$

In [5]:
def F_function(T_strong, V_odd, phase_list, q0, flag):
    
    pgrid[Np] = q0
    
    
    def sum_1(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2):

        sum = 0

        for k1 in range(Np):
            sum+=  ( 
                    2 * mu
                    * (
                        T_strong[s1,l1,j1,sk,lk,jk, i1, k1] * pgrid[k1]**2 * V_odd[sk,lk,jk,s2,l2,j2, k1, i2] 
                        -
                        T_strong[s1,l1,j1,sk,lk,jk, i1, Np] * q0**2 * V_odd[sk,lk,jk,s2,l2,j2, Np, i2]
                        )
                    * wp[k1]
                )/( q0**2 - pgrid[k1]**2 )

        return sum
    
    
    def I_lambda():

        return  ( 
                    (2 * q0)**(-1)
                    * log( (p3 + q0)/ (p3 - q0) )
                )
    
            
    def term_1(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2):

        return  (
                    2 * mu * q0**2 * I_lambda()
                    *  T_strong[s1,l1,j1,sk,lk,jk, i1, Np] *  V_odd[sk,lk,jk,s2,l2,j2, Np, i2]
                )
    
    def term_2(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2):

        return  ( 
                    -1j * mu * pi * q0 
                    *  T_strong[s1,l1,j1,sk,lk,jk, i1, Np] *  V_odd[sk,lk,jk,s2,l2,j2, Np, i2]
                )
            
    # T_ans = T_s . G0 . V_odd 
            
    T_ans = {}
            
    for key in T_strong:
        
        [s1,l1,j1,s2,l2,j2, i1, i2] = key
        
        
        
    
        # for F1, F2, F3 we only need T_(np,np)
        # for F3A we need T_{i, np}, since F3A = F2, flag = 1 for F2 
    
        if flag == 0:
            if [i1,i2]!=[Np,Np]:
                continue
        else:
            if i2!= Np:
                continue
        ans = 0

        for elem in phase_list:

            [sk,lk,jk] = elem

            sum__1  = sum_1(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2)
            term__1 = term_1(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2)
            term__2 = term_2(s1,l1,j1,s2,l2,j2,sk,lk,jk,i1,i2)

            ans     += sum__1 + term__1 + term__2

        T_ans[s1,l1,j1,s2,l2,j2, i1, i2]    = ans
            
    return T_ans
        
        

## method 1

In [6]:
def cpv_method1(phase_list, lam, q0, Vtensor_s, Vtensor_cpv):
    
    index_s = 0
    
    [smat_s, Tmat_s, Tdict_s] = phase_shift_calc(phase_list, q0, Vtensor_s, lam, index_s)
        
    index_cpv = 1
    Vtensor_lam_cpv = pot_lambda(Vtensor_cpv,lam,index_cpv)
    
    # F0: V_odd(p',p)
    F0 = deepcopy(Vtensor_lam_cpv)
    
    # F1: T_s . G . V_odd
    flag = 0
    F1 = F_function(Tdict_s, Vtensor_lam_cpv, phase_list, q0,flag)

    # F2: V_odd . G . T_s
    flag = 1
    F2 = F_function(Vtensor_lam_cpv, Tdict_s, phase_list, q0,flag)

    # F3A: 
    F3A = deepcopy(F2)
    
    # F3: T_s . G . V_odd . G . T_s
    flag = 0
    F3 = F_function(Tdict_s, F3A, phase_list, q0,flag)
    
    Tdict_cpv = {}
    
    for key in F1:
        Tdict_cpv[key] =  F0[key] + F1[key] + F2[key] + F3[key]
        
    phase_len = len(phase_list)
        
    S_cpv = np.matrix(np.empty([phase_len,  phase_len], dtype = complex))

    for i_1 in range(phase_len):
        for j_1 in range(phase_len):

            [s1,l1,j1] = phase_list[i_1]
            [s2,l2,j2] = phase_list[j_1]

            S_cpv[i_1,j_1] = ( 
                                kron(i_1, j_1)
                                - (
                                    1j * pi * m_n * q0  
                                    * (
                                        Tdict_s[s1,l1,j1,s2,l2,j2,Np,Np]
                                        +
                                        Tdict_cpv[s1,l1,j1,s2,l2,j2,Np,Np]
                                      )
                                  )
                              )

    return S_cpv

## method 2


In [7]:
def cpv_method2(phase_list, lam, q0, Vtensor_m2):

    index_m2 = 2
    [smat_m2, Tmat_m2, Tdict_m2] = phase_shift_calc(phase_list, q0, Vtensor_m2, lam, index_m2)

    return smat_m2

In [8]:
# # Example

# phase_list = phase_dict[0]
# p_wave = [phase_name(*phase) for phase in phase_list]
# print(phase_list,'==',p_wave,'\n\n')

# elab    = 10.
# e0_true  = 0.01
# lam = 2
# q0 = sqrt( elab * m_n * .5)

# [Vtensorq0_strong, Vtensorq0_odd,Vtensorq0_even_odd] = method_input(phase_list,q0)

# # to set the counter term to zero
# cpv_c0_choice = 3

# print('Method 1:\n')
# smat_m1 = cpv_method1(phase_list, lam, q0, Vtensorq0_strong, Vtensorq0_odd)
# round_n(smat_m1,expo=True)

# print('\nMethod 2:\n')
# smat_m2 = cpv_method2(phase_list, lam, q0, Vtensorq0_even_odd)
# round_n(smat_m2,expo=True)

# print('\nDifference:\n')
# round_n(smat_m1-smat_m2,expo=True)

## counter term calculation

In [9]:
def cpv_ct_calc_j0():
    
    folder = 'c0/cpv_'

    
    phase_list   = [[0,0,0],[1,1,0]]

    elab     = 10.
    e0_true_list  = [0.01,.11,-.09]
    q0       = sqrt( elab * m_n * .5)
    
    # generating filename
    filename = ''
    for i in phase_list:
        filename += phase_name(*i)+'_'
    
    name = folder + filename + '.csv'
    
    if exists(name):
        
        pd1 = pd.read_csv(name, index_col=None)
        d1 = pd1.to_dict('list')
        
        return d1
    
    else:
    
        # V_s + V_cpv : index 2

        Vtensor0_i2 = Vtensor_calc(phase_list,2)
        Vtensor_i2  = Vtensor_q0_calc(phase_list,Vtensor0_i2,q0,2)

        # V_s + V_odd_contact : index 3

        Vtensor0_i3 = Vtensor_calc(phase_list,3)
        Vtensor_i3  = Vtensor_q0_calc(phase_list,Vtensor0_i3,q0,3)

        print('Fitting Starts...\n\n')

        d1 = {}
        d1['lam']=[]
        col_list = []
        for e0_true in e0_true_list:
            
            col_name = 'e0_true = {:.2f}'.format(e0_true)
            col_list.append(col_name)
            d1[col_name] = []
        
        d1['not renorm'] =[]
        
        
        print('\n   {:8}   '.format('lam'),end='')
        for col in col_list:
            print('{:17}   '.format(col),end='')
        print()
        
#************************************
#************************************

        # we use method 2 for the calculation, since it is slower. 
        # it has been verified that both method 1 and method 2 give same results
        for lam in lambda_values:

            # V_s + V_cpv
            smatA = cpv_method2(phase_list, lam, q0, Vtensor_i2)
            [d0A, d2A, e0A] = phase_shift_cpv(smatA)

            # V_s + V_odd_contact
            smatB = cpv_method2(phase_list, lam, q0, Vtensor_i3)
            [d0B, d2B, e0B] = phase_shift_cpv(smatB)
            
            print('{:8.2f}   '.format(lam),end='')
            
            for i  in range(len(e0_true_list)):
                
                e0_true = e0_true_list[i]
                
                # e0_true = e0_odd + c0 * e0_odd_contact
                c0 = (e0_true - e0A)/e0B
                
                print('{:17.5f}   '.format(c0),end='')
                d1[col_list[i]].append(c0)
            
            print()
            
            d1['lam'].append(lam)
            d1['not renorm'].append(0)\
            
        pd1 = pd.DataFrame(d1)
        pd1.to_csv(name, index=False)
        
        return d1




## counterm calculation 

In [14]:
print('\nCalculating counter term CP-odd C0(lam) for four values of esp_exp.\n')

if exists('c0/cpv_1S0_3P0_.csv'):
    pd1 = pd.read_csv('c0/cpv_1S0_3P0_.csv')
else:
    d2 =cpv_ct_calc_j0()
    pd1 = pd.DataFrame(d2)

print(pd1.head())


Calculating counter term CP-odd C0(lam) for four values of esp_exp.

   lam  e0_true = 0.01  e0_true = 0.11  e0_true = -0.09  not renorm
0  2.0        0.888454        1.067591         0.709317           0
1  2.1        0.875090        1.050320         0.699861           0
2  2.2        0.866117        1.038843         0.693391           0
3  2.3        0.861897        1.033523         0.690272           0
4  2.4        0.862963        1.034950         0.690977           0
